# Sequencial

In [13]:
from pettingzoo.butterfly import pistonball_v6

# Creates env
env = pistonball_v6.env(render_mode="human")
env.reset(seed=42)


for agent in env.agent_iter():
    # Get observation and reward of the agent
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # Randomly selected action from the action space
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

# Parallel

In [4]:
from pettingzoo.butterfly import pistonball_v6

env = pistonball_v6.parallel_env(render_mode="human", n_pistons = 20)
observations, infos = env.reset()



while env.agents:
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}

    observations, rewards, terminations, truncations, infos = env.step(actions)

env.close()

# Wrapping Environment for Fully Centralized Control

In [6]:
import gymnasium
from gymnasium import spaces
import numpy as np



class SingleAgentWrapperEnv(gymnasium.Env) :
    """
    This wrapper permits to create a gymnasium env where the action space is the cartesian product of agents' action space
    and the observation space is the cartesian product of agents' observation space.

    This permits to train a single "super-agent" which will receive all observations and distribute actions that has to be made by all "sub-agents".

    WARNING : USE THIS WRAPPER ONLY IF YOU FOLLOW ALL ASSUMPTIONS BELOW !!!
     - The env is a pettingzoo env with the parallel API (ParallelEnv) (can be wrapped)
     - All agents of the env has a Box action space and a Box observation space
     - All agents of the env must have the same bounds, dtypes and a shape of (1,) for the Box representing their action space
     - All agents of the env must have the same bounds, dtypes and a 1D shape of same size for the Box space representing their observation space


    Do not forget to use env.reset() before creating SingleAgentWrapperEnv(env) to properly initialize all attributes of the env
    """

    def __init__(self, env, options = None) :
        super(SingleAgentWrapperEnv, self).__init__()

        self.env = env
        self.agents = self.env.unwrapped.agents
        self.nb_agent = len(self.agents)


        # Creating observation_space
        temp_space = self.env.observation_space(self.agents[0])
        low_bound = min(temp_space.low) # Not optimized this using min instead of taking the lowest value for each element grows the observation_space
        high_bound = max(temp_space.high) # Same here with the max
        dtype = temp_space.dtype

        shape = (self.nb_agent * temp_space.shape[0],)
        self.observation_space = spaces.Box(np.full(shape, low_bound), np.full(shape, high_bound), shape, dtype)

        
        # Creating action_space
        temp_space = self.env.action_space(self.agents[0])
        low_bound = temp_space.low # Lowest value an action of an agent can take
        high_bound = temp_space.high # Highest value an action of an agent can take
        dtype = temp_space.dtype # The exact type of an action
        
        shape = (self.nb_agent,) # A vector container at index i the action made by self.env.agents[i]
        self.action_space = spaces.Box(np.full(shape, low_bound), np.full(shape, high_bound), shape, dtype)


    def reset(self, seed = None, options = None) :
        super().reset(seed = seed)
        observations, infos = self.env.reset(seed, options)

        obs = np.array([], dtype = self.observation_space.dtype)
        for i in range(self.nb_agent) :
            obs = np.append(obs, observations[self.agents[i]]) # Concatenation of all observations
        return obs, infos


    def step(self, action) :
        """
        The action in input is a vector containing actions of each agents
        """
        dict_actions = {self.agents[i]: np.array(object = [action[i]], dtype=np.float32) for i in range(self.nb_agent)}
        
        observations, rewards, terminations, truncations, infos = self.env.step(dict_actions)

        
        obs = np.array([], dtype = self.observation_space.dtype)
        for i in range(self.nb_agent) :
            obs = np.append(obs, observations[self.agents[i]]) # Concatenation of all observations

        
        reward = 0
        termination = False
        truncation = False
        for agent in self.agents :
            # Reward is the mean of all agents' reward
            reward += rewards[agent]
            reward = reward / self.nb_agent

            # Episode ends as soon as it ends for one agent
            if terminations[agent] :
                termination = True

            if truncations[agent] :
                truncation = True

        return obs, reward, termination or len(self.agents) != self.nb_agent, truncation, infos


### Use Grey Scale Image as observation and flatten it

In [2]:
from pettingzoo.butterfly import pistonball_v6

env = pistonball_v6.parallel_env()
env.reset()


from supersuit import color_reduction_v0
from supersuit import resize_v1
from supersuit import flatten_v0

grey_scale_env = color_reduction_v0(env, mode='full') # This changes observations as grey scale images
grey_scale_env = resize_v1(grey_scale_env, 114, 30) # Divide by 4 quality of each image observed by each agent
grey_scale_env = flatten_v0(grey_scale_env) # Flatten to 1D to make it compatible with our wrapper

env.close()

/home/enzo/Documents/Depots/customEnv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### Testing the wrapped environment

In [3]:
from pettingzoo.butterfly import pistonball_v6
from stable_baselines3.common.env_checker import check_env
import supersuit

env = pistonball_v6.parallel_env()
env.reset()

grey_scale_env = supersuit.color_reduction_v0(env, mode='full') # This changes observations as grey scale images
grey_scale_env = supersuit.resize_v1(grey_scale_env, 57, 15) # Divide by 8 size of each image observed by each agent
grey_scale_env = supersuit.flatten_v0(grey_scale_env) # Flatten to 1D to make it compatible with our wrapper
grey_scale_env = supersuit.frame_stack_v1(grey_scale_env, 3) # Observations are now the past 3 observations (so pistons can observe in which direction the ball moves)


wrappedEnv = SingleAgentWrapperEnv(grey_scale_env)

check_env(wrappedEnv)

while env.aec_env.agents :
    # this is where you would insert your policy
    actions = [env.action_space(env.agents[0]).sample()[0] for agent in env.aec_env.agents]
    
    observations, rewards, terminations, truncations, infos = wrappedEnv.step(actions)

env.close()
print(wrappedEnv.observation_space)
print("Test of the Wrapped Environment is done.")

Box(0, 255, (51300,), uint8)
Test of the Wrapped Environment is done.


### Initializing PPO agent on wrapped environment

In [7]:
from pettingzoo.butterfly import pistonball_v6
import supersuit

# Creating env
env = pistonball_v6.parallel_env(n_pistons = 10)
env.reset()


grey_scale_env = supersuit.color_reduction_v0(env, mode='full') # This changes observations as grey scale images
grey_scale_env = supersuit.resize_v1(grey_scale_env, 57, 15) # Divide by 8 size of each image observed by each agent
grey_scale_env = supersuit.flatten_v0(grey_scale_env) # Flatten to 1D to make it compatible with our wrapper
grey_scale_env = supersuit.frame_stack_v1(grey_scale_env, 3) # Observations are now the past 3 observations (so pistons can observe in which direction the ball moves)


wrappedEnv = SingleAgentWrapperEnv(grey_scale_env)


# Initializing PPO
from stable_baselines3 import PPO

model = PPO("MlpPolicy", wrappedEnv)
print("The PPO model has been initialized.")

The PPO model has been initialized.


In [3]:
print("Neural networks used by model.")
print(model.policy)

Neural networks used by model.
ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=51300, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=51300, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=20, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)


### Train PPO agent

In [ ]:
model.learn(total_timesteps=100000)

env.close()

### Save PPO agent

In [13]:
model.save("./models/crossProductPPO.zip")

/media/thewalder/Big Disk/Travail/customEnv/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


### Import PPO agent

In [8]:
model = model.load("./models/crossProductPPO.zip")

### Test PPO agent

In [15]:
from stable_baselines3.common.env_util import make_vec_env
from pettingzoo.butterfly import pistonball_v6
import supersuit

# Creating env
env = pistonball_v6.parallel_env(render_mode="human", n_pistons=10)
env.reset()


grey_scale_env = supersuit.color_reduction_v0(env, mode='full') # This changes observations as grey scale images
grey_scale_env = supersuit.resize_v1(grey_scale_env, 57, 15) # Divide by 8 size of each image observed by each agent
grey_scale_env = supersuit.flatten_v0(grey_scale_env) # Flatten to 1D to make it compatible with our wrapper
grey_scale_env = supersuit.frame_stack_v1(grey_scale_env, 3) # Observations are now the past 3 observations (so pistons can observe in which direction the ball moves)


wrappedEnv = SingleAgentWrapperEnv(grey_scale_env)

vec_env = make_vec_env(lambda : wrappedEnv, n_envs=1)


# Get first observation
obs = vec_env.reset()

done = False
while not done :
    action, _states = model.predict(obs, deterministic = True)
    obs, reward, done, info = vec_env.step(action)


env.close()

# Parameter Sharing

All pistons have the same goal, same rewards and same observations.

Instead of training each pistons or training a single agent controlling all pistons, we could consider all pistons being the same one.

In this case we would use all observations and rewards from all pistons to train only a single piston.
And then treat all pistons as being a copy of this single piston.


In fact parameter sharing cannot work here based on how rewards are distributed in this environment : same reward for each agent. Thus useless pistons (and actions they made) will have the same impact (even a bigger impact since they are more than usefull ones) as usefull pistons which make the ball going to the left wall.

### Initialize PPO agent

In [2]:
from pettingzoo.butterfly import pistonball_v6
import numpy as np
import supersuit

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env


env = pistonball_v6.parallel_env(render_mode = None, n_pistons=10) # We use AEC env
env.reset()


env = supersuit.color_reduction_v0(env, mode="full")
env = supersuit.resize_v1(env, 114, 30)
env = supersuit.reshape_v0(env, env.observation_space(env.unwrapped.agents[0]).shape + (1,))
env = supersuit.frame_stack_v1(env, 5)

env = supersuit.pettingzoo_env_to_vec_env_v1(env)
env = supersuit.concat_vec_envs_v1(env, 1, num_cpus=1, base_class="stable_baselines3")

print(env.observation_space)


#model = PPO("CnnPolicy", env)
model = PPO("MlpPolicy", env)
print("The PPO model has been initialized.")

Box(0, 255, (30, 114, 5), uint8)


/home/enzo/Documents/Depots/customEnv/lib/python3.12/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:78: UserWarning: The `render_mode` attribute is not defined in your environment. It will be set to None.
  warnings.warn("The `render_mode` attribute is not defined in your environment. It will be set to None.")


The PPO model has been initialized.


In [3]:
print("Neural networks used by model.")
print(model.policy)

Neural networks used by model.
ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=17100, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=17100, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)


### Train PPO agent

In [4]:
model.learn(total_timesteps=1000)

env.close()

### Test PPO Agent

In [6]:
from stable_baselines3.common.env_util import make_vec_env
import numpy as np

import supersuit


# Creating env
env = pistonball_v6.parallel_env(render_mode = "human", n_pistons=10) # We use AEC env
env.reset()


env = supersuit.color_reduction_v0(env, mode="full")
env = supersuit.resize_v1(env, 114, 30)
env = supersuit.reshape_v0(env, env.observation_space(env.unwrapped.agents[0]).shape + (1,))
env = supersuit.frame_stack_v1(env, 5)

env = supersuit.pettingzoo_env_to_vec_env_v1(env)
vec_env = supersuit.concat_vec_envs_v1(env, 1, num_cpus=1, base_class="stable_baselines3")


#vec_env = make_vec_env(lambda : env, n_envs=1)


# Get first observation
obs = vec_env.reset()

done = np.array([False])
while not done.any() :
    action, _states = model.predict(obs, deterministic = True)
    obs, reward, done, info = vec_env.step(action)


env.close()

# RLlib

### Register Env

In [1]:
import ray
from pettingzoo.butterfly import pistonball_v6
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
import supersuit

ray.init()


env_name = "pistonball_v6"

def env_creator(args) :
    env = pistonball_v6.parallel_env(n_pistons=20)
    env = supersuit.color_reduction_v0(env, mode='full') # image to black and white
    env = supersuit.resize_v1(env, 84, 84) # Reduce size of image
    env = supersuit.reshape_v0(env, env.observation_space('piston_0').shape + (1,))
    return env


register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

/home/enzo/Documents/customEnv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
2025-11-20 16:50:33,958	INFO worker.py:2012 -- Started a local Ray instance.
/home/enzo/Documents/customEnv/lib/python3.12/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
(MultiAgentEnvRunner pid=275336) /home/enzo/Documents/customEnv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://se

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. Lease ID: 000000001afceac94d16bb570e1144be8c386d3ef25f54486d26b374f0533b74 Worker ID: d390dcba3c3fd3f7a0d032c869bd0289398c69e8b578f101032d8b9f Node ID: 90904ca1f7aa5f1819a8ce8ff895d8df02244ce1596ed849fb971d7c Worker IP address: 134.206.153.65 Worker port: 35551 Worker PID: 275336 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


(MultiAgentEnvRunner pid=275339) /home/enzo/Documents/customEnv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
(MultiAgentEnvRunner pid=275339)   from pkg_resources import resource_stream, resource_exists
(MultiAgentEnvRunner pid=275339) DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


In [2]:
env = env_creator("")
env.reset()
print(env.observation_space('piston_0').shape)

(84, 84, 1)


### Initialize Model

In [3]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec


config = (
    PPOConfig()
    .environment(env=env_name)
    .multi_agent(policies = env.unwrapped.agents, policy_mapping_fn = lambda agent_id, episode, **kwargs : agent_id)
)


config.validate()
print(config.is_multi_agent)

algo = config.build_algo()

"""
    .rl_module(
        rl_module_spec = MultiRLModuleSpec(rl_module_specs={
            agent_id : RLModuleSpec() for agent_id in env.unwrapped.agents
        })
    )
"""

2025-11-20 16:50:43,074	WARNING algorithm_config.py:5053 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/home/enzo/Documents/customEnv/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:526: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/enzo/Documents/customEnv/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress thi

True


2025-11-20 16:50:47,506	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


'\n    .rl_module(\n        rl_module_spec = MultiRLModuleSpec(rl_module_specs={\n            agent_id : RLModuleSpec() for agent_id in env.unwrapped.agents\n        })\n    )\n'

### Train Model

In [4]:
algo.train()

[2025-11-20 16:51:08,150 E 275136 275334] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
2025-11-20 16:51:52,878	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2025-11-20 16:51:54,155	ERROR actor_manager.py:973 -- Ray error (The actor 074e422160230821ba80615901000000 is unavailable: The actor is temporarily unavailable: IOError: The actor was restarted. The task may or may not have been executed on the actor.), taking actor 1 out of service.
2025-11-20 16:51:54,156	ERROR actor_manager.py:973 -- Ray error (The actor caa19bc5e6d43895a1f7222401000000 is unavailable: The actor is temporarily unavailable: IOError: The actor was restart

{'timers': {'training_iteration': 60.00750333699398,
  'restore_env_runners': 3.207288682460785e-05,
  'training_step': 60.007248636102304,
  'env_runner_sampling_timer': 60.00712985591963},
 'num_training_step_calls_per_iteration': 1,
 'num_env_steps_sampled_lifetime': 0,
 'fault_tolerance': {'num_healthy_workers': 0,
  'num_remote_worker_restarts': 0},
 'env_runner_group': {'actor_manager_num_outstanding_async_reqs': 0},
 'done': False,
 'training_iteration': 1,
 'trial_id': 'default',
 'date': '2025-11-20_16-51-54',
 'timestamp': 1763653914,
 'time_this_iter_s': 61.28780245780945,
 'time_total_s': 61.28780245780945,
 'pid': 275136,
 'hostname': 'smac-Precision-Tower-3620',
 'node_ip': '134.206.153.65',
 'config': {'exploration_config': {},
  'extra_python_environs_for_driver': {},
  'extra_python_environs_for_worker': {},
  'placement_strategy': 'PACK',
  'num_gpus': 0,
  '_fake_gpus': False,
  'num_cpus_for_main_process': 1,
  'eager_tracing': True,
  'eager_max_retraces': 20,
  't

In [5]:
algo.evaluate()

ValueError: <ray.rllib.env.multi_agent_env_runner.MultiAgentEnvRunner object at 0x79d93822b080> doesn't have an env! Can't call `sample()` on it.

In [7]:
algo.stop()

ray.shutdown()

# Test

In [1]:
"""Uses Ray's RLlib to train agents to play Pistonball.

Author: Rohan (https://github.com/Rohan138)
"""

import os
from pathlib import Path

import ray
import supersuit as ss
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.tune.registry import register_env
from torch import nn

from pettingzoo.butterfly import pistonball_v6


class CNNModelV2(TorchModelV2, nn.Module):
    def __init__(self, obs_space, act_space, num_outputs, *args, **kwargs):
        TorchModelV2.__init__(self, obs_space, act_space, num_outputs, *args, **kwargs)
        nn.Module.__init__(self)
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, [8, 8], stride=(4, 4)),
            nn.ReLU(),
            nn.Conv2d(32, 64, [4, 4], stride=(2, 2)),
            nn.ReLU(),
            nn.Conv2d(64, 64, [3, 3], stride=(1, 1)),
            nn.ReLU(),
            nn.Flatten(),
            (nn.Linear(3136, 512)),
            nn.ReLU(),
        )
        self.policy_fn = nn.Linear(512, num_outputs)
        self.value_fn = nn.Linear(512, 1)

    def forward(self, input_dict, state, seq_lens):
        model_out = self.model(input_dict["obs"].permute(0, 3, 1, 2))
        self._value_out = self.value_fn(model_out)
        return self.policy_fn(model_out), state

    def value_function(self):
        return self._value_out.flatten()


def env_creator(args):
    env = pistonball_v6.parallel_env(
        n_pistons=20,
        time_penalty=-0.1,
        continuous=True,
        random_drop=True,
        random_rotate=True,
        ball_mass=0.75,
        ball_friction=0.3,
        ball_elasticity=1.5,
        max_cycles=125,
    )
    env = ss.color_reduction_v0(env, mode="B")
    env = ss.dtype_v0(env, "float32")
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.normalize_obs_v0(env, env_min=0, env_max=1)
    env = ss.frame_stack_v1(env, 3)
    return env


if __name__ == "__main__":
    ray.init()

    env_name = "pistonball_v6"

    def _make_rllib_env(config):
        base = env_creator(config)
        wrapped = ParallelPettingZooEnv(base)
        wrapped._agent_ids = set(getattr(base, "possible_agents", []))
        return wrapped

    register_env(env_name, _make_rllib_env)

    ModelCatalog.register_custom_model("CNNModelV2", CNNModelV2)

    config = (
        PPOConfig()
        .environment(
            env=env_name,
            clip_actions=True,
            disable_env_checking=True,
        )
        .training(
            train_batch_size=512,
            lr=2e-5,
            gamma=0.99,
            lambda_=0.9,
            use_gae=True,
            clip_param=0.4,
            grad_clip=None,
            entropy_coeff=0.1,
            vf_loss_coeff=0.25,
            num_sgd_iter=10,
        )
        .debugging(log_level="ERROR")
        .framework(framework="torch")
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
        .multi_agent(
            policies = "learning_policy",
            policy_mapping_fn = 'learning_policy',
            policies_to_train=["learning_policy"]
        )
    )

    storage_uri = (Path("~/ray_results") / env_name).expanduser().resolve().as_uri()

    tune.run(
        "PPO",
        name="PPO",
        stop={"timesteps_total": 5000000 if not os.environ.get("CI") else 50000},
        checkpoint_freq=10,
        storage_path=storage_uri,
        config=config.to_dict(),
    )

/home/enzo/Documents/Depots/customEnv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
2025-11-14 16:35:45,612	INFO worker.py:2012 -- Started a local Ray instance.
/home/enzo/Documents/Depots/customEnv/lib/python3.12/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
2025-11-14 16:35:49,137	WARNING deprecation.py:50 -- DeprecationWarning: `config.training(num_sgd_iter=..)` has been deprecated. Use `config.training(num_epochs=..)` i

ValueError: `policies` must be dict mapping PolicyID to PolicySpec OR a set/tuple/list of PolicyIDs!

(pid=gcs_server) [2025-11-14 16:36:13,677 E 446246 446246] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(raylet) [2025-11-14 16:36:15,545 E 446344 446344] (raylet) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(pid=446398) [2025-11-14 16:36:18,704 E 446398 446463] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
